## Import Necessary Packages

In [1]:
import os
import glob
import numpy as np
import xarray as xr
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

## User-Defined Fields

In [2]:
author = # 'your name'
email  = # 'your email'
path   = # '/directory/where/small/or/large/domain/files/are/located/'
simset = 'rcemip-small' # or 'rcemip-large'
dirout = 'massflux'
cases  = ['nz_32','nz_64','nz_128','nz_256']
today  = datetime.today().strftime('%Y-%m-%d')

In [3]:
path = path+simset
if simset == 'rcemip-small':
    dx = 780
    outstr = '_99km_300K_'
    nxny   = '_128_'
else:
    cases = cases[:2]
    dx = 3_000
    outstr = '_1536km_300K_'
    nxny   = '_512_'

## Calculate and Save Mass Flux

Calculate density following $\rho = \frac{p}{RT}$, then calculate mass flux of each column following $M = \rho w$. Store $M$ in Xarray.Datasets by case, and save as netCDF files to ```dirout```.

In [4]:
for i,case in enumerate(cases):
    ## Check to See if Directory Exists
    try: os.mkdir(path+case+'/OUT_3D/'+dirout)
    except: print('Error: directory already exists')
    print('Working on '+case+'...')    
    ## List Files
    pfile   = glob.glob(path+case'/OUT_3D/p/*.nc')[0]
    wfiles  = np.sort(glob.glob(path+case+'/OUT_3D/W/*.nc'))
    tfiles  = np.sort(glob.glob(path+case+'/OUT_3D/TABS/*.nc'))
    ppfiles = np.sort(glob.glob(path+case+'/OUT_3D/PP/*.nc'))
    for j in range(len(wfiles)):
        timestr = wfiles[i][-13:-3]
        ## Check to See if File Exists
        filename = (path+case+'/OUT_3D/'+dirout+'/RCE_'+simset+outstr+case+nxny+dirout+'_'+timestr+'.nc')
        if os.path.exists(filename): 
            print('Error: file already exists and will be skipped')
            continue
        ## Define Variables
        p_background = xr.open_dataset(gpfile[j]).p*100 # from hPa to Pa
        p = xr.load_dataset(ppfiles[j]).PP + p_background
        w = xr.load_dataset(wfiles[j]).W
        T = xr.load_dataset(tfiles[j]).TABS
        ## Calculate Density
        R = 287
        rho = p/(R*T)
        del T
        del p
        ## Calculate Mass Flux
        M = rho*w
        M.attrs['long_name'] = 'Mass flux'
        M.attrs['units'] = '$kg m^{-2} s^{-1}$'
        del rho
        del w
        ## Create Dataset and Save as a NetCDF File
        print('Creating and saving dataset to: '+filename)
        ds = xr.Dataset(data_vars=dict(M=M),attrs=dict(history='Calculated on '+today+' by '+author+': '+email))
        ds.to_netcdf(filename)
        del M
        del ds